# 1 Introduccion
En matemática se llama multiplicacion o producto de matrices a la operación  de composición efectuada entre dos matrices. Esta multiplicación no cumple con la propiedad de conmutatividad.

Suponiendo que tenemos una matriz producto donde cada elemento ocupa la posion ik, esta matriz se obtiene sumando los productos que resulta de multiplicar los elemetnos de la fila i de la primera matriz por los elementos de la columna k de la segunda matriz.

Dos matrices solo se podran multiplcar si se cumple que el número de columna de la primera matriz sea igual al número de filass  de la segunda. En esta caso se dice que las matrices son enlazadas

Resolveremos la multiplicación de matrices con un algoritmo que se ejecutará de manera secuencia y luego con OpenMP para ver sus ventajas y desventajas.

Para ejecutar el programa hay que tener en cuenta los siguientes parametros:

./producto_matrices a b c d

*   a: Entero que representa la cantidad de columnas de la matriz A y la cantidad de filas de la matriz B
*   b: Entero que representa la cantidad de filas de la matriz A
*   c: Entero que representa la cantidad de columnas de la matriz B
*   d: Ciclos












# 2 Armado del ambiente

No son necesarios comandos previos para la ejecución correcta del desarrollo.

# 3 desarrollo



In [ ]:
code = """

#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP   

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_SEC      2
#define HTH_OMP      3

#define COLUMNAS          3
#define FILAS             3

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  int i,c;
  TIEMPO_INI( HTH_TOTAL )

  // Leo los parametros.
  if( argc != 5 )
  {
      std::cerr<< " Error en los parametros de indicar: (Numero de columnas de A y filas de B),(Numero de filas de B),(Numero de columnas de B), (ciclos ejecucion)."<<argc<<std::endl;
      exit( -1 );
  }

  int cantidad_N = atoi( argv[1] );
  int filas_A  = atoi( argv[2] );
  int columnas_B = atoi( argv[3] );
  int ciclos     = atoi( argv[4] );

  // Defino la memoria de las matrices.
  std::vector<std::vector<int>> matriz_a(filas_A,std::vector<int>(cantidad_N));
  std::vector<std::vector<int>> matriz_b(cantidad_N,std::vector<int>(columnas_B));
  std::vector<std::vector<int>> matriz_r(filas_A,std::vector<int>(columnas_B));
  

  //llenamos la matriz a de valores aleaotorios
  for (int i=0;i<filas_A;i++)
  {
    for(int j=0;j<cantidad_N;j++)
    {
      matriz_a[i][j] = (std::rand() % 10);
    }
  }

  //llenamos la matriz b de valores aleaotorios
  for (int i=0;i<cantidad_N;i++)
  {
    for(int j=0;j<columnas_B;j++)
    {
      matriz_b[i][j] = (std::rand() % 10);
    }
  }

    
  // Multiplicacion de matrices de manera secuencial.

  TIEMPO_INI( HTH_SEC )

  for (int z = 0; z < columnas_B; z++) {
    for (int i = 0; i < filas_A; i++) {
        int suma = 0;
        for (int j = 0; j < cantidad_N; j++) {
            suma += matriz_a[i][j] * matriz_b[j][z];
          }
            matriz_r[i][z] = suma;
        }
    }

  TIEMPO_FIN( HTH_SEC )
  
  // Con OpenMP.

  TIEMPO_INI( HTH_OMP )

for(c=0;c<ciclos;c++)
  {
  #pragma omp parallel for
  for (int z = 0; z < columnas_B; z++) {
    //#pragma omp parallel for
    for (int i = 0; i < filas_A; i++) {
        int suma = 0;
        //#pragma omp parallel for
        for (int j = 0; j < cantidad_N; j++) {
            suma += matriz_a[i][j] * matriz_b[j][z];
          }
            matriz_r[i][z] = suma;
        }
    }
}
  TIEMPO_FIN( HTH_OMP )

// Muestro las matrices
std::cout<<"------------------------------------------" <<std::endl;
std::cout<<"Matriz A :" <<std::endl;
std::cout<<"------------------------------------------" <<std::endl;
    for (int i = 0; i < filas_A; i++) {
        for (int j = 0; j < cantidad_N; j++) {
             std::cout<<matriz_a[i][j]<<" ";
        }
       /* std::cout<<<<std::endl; */
       std::cout<<" "<<std::endl; 
    }

std::cout<<"------------------------------------------" <<std::endl;
std::cout<<"Matriz B :" <<std::endl;
std::cout<<"------------------------------------------" <<std::endl;
    for (int i = 0; i < cantidad_N; i++) {
        for (int j = 0; j < columnas_B; j++) {
             std::cout<<matriz_b[i][j]<<" ";
        }
       /* std::cout<<<<std::endl; */
       std::cout<<" "<<std::endl; 
    }
std::cout<<"------------------------------------------" <<std::endl;
std::cout<<"Matriz R :" <<std::endl;
std::cout<<"------------------------------------------" <<std::endl;
    for (int i = 0; i < filas_A; i++) {
        for (int j = 0; j < columnas_B; j++) {
             std::cout<<matriz_r[i][j]<<" ";
        }
       /* std::cout<<<<std::endl; */
       std::cout<<" "<<std::endl; 
    }
std::cout<<"------------------------------------------" <<std::endl;
  TIEMPO_FIN( HTH_TOTAL )

 std::cout<<"Valores Reales  :" <<std::endl;
 std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
 std::cout<<"Tiempo  Sec  : "<<TIEMPO_GET(HTH_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo  Omp  : "<<TIEMPO_GET(HTH_OMP)<<" [ms]"<<std::endl;
 std::cout<<std::endl;
 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_SEC)<<" / "<<TIEMPO_GET(HTH_OMP)<<" = "<<TIEMPO_GET(HTH_SEC)/TIEMPO_GET(HTH_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_SEC)/TIEMPO_GET(HTH_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_SEC)<<" = "<<TIEMPO_GET(HTH_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_OMP)<<" - "<<TIEMPO_GET(HTH_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_OMP))-TIEMPO_GET(HTH_SEC)<<std::endl;


 std::cout<<std::endl;
 std::cout<<"Valores Ideal: "<<std::endl;
 TIEMPO_GET(HTH_OMP) = TIEMPO_GET(HTH_SEC) / 2;
 std::cout<<"Tiempo  Sec  : "<<TIEMPO_GET(HTH_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo  Omp  : "<<TIEMPO_GET(HTH_OMP)<<" [ms]"<<std::endl;

 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_SEC)<<" / "<<TIEMPO_GET(HTH_OMP)<<" = "<<TIEMPO_GET(HTH_SEC)/TIEMPO_GET(HTH_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_SEC)/TIEMPO_GET(HTH_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_SEC)<<" = "<<TIEMPO_GET(HTH_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_OMP)<<" - "<<TIEMPO_GET(HTH_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_OMP))-TIEMPO_GET(HTH_SEC)<<std::endl;


}
// ----------------------------------------------------------------------------

"""
text_file = open("code.cpp", "w")
text_file.write(code)
text_file.close()

In [ ]:
!g++ -o producto_matrices -fopenmp code.cpp

In [ ]:
%env OMP_NUM_THREADS=2
!./producto_matrices 3 2 2 20

env: OMP_NUM_THREADS=2
------------------------------------------
Matriz A :
------------------------------------------
3 6 7  
5 3 5  
------------------------------------------
Matriz B :
------------------------------------------
6 2  
9 1  
2 7  
------------------------------------------
Matriz R :
------------------------------------------
86 61  
67 48  
------------------------------------------
Valores Reales  :
Tiempo TOTAL     : 0.268221 [ms]
Tiempo  Sec  : 0.000953674 [ms]
Tiempo  Omp  : 0.129938 [ms]

SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 0.000953674 / 0.129938 = 0.00733945
Eficiencia       : SpeedUp/nro procesadores            : 0.00733945 / 2 = 0.00366972
Coste Sec        : nro procesadores*Tiempo             : 1 * 0.000953674 = 0.000953674
Coste Omp        : nro procesadores*Tiempo             : 2 * 0.129938 = 0.259876
Funcion Overhead : Coste Omp - tiempo Secuencial       : 0.259876 - 0.000953674 = 0.258923

Valores Ideal: 
Tiempo  Sec  : 0.000953674

# 4 Tabla de pasos
Procesador | Función | Detalle
---------- | -------   | -------
CPU        | include   | Importa las bibliotecas necesarias para la ejecucion
CPU        | gettimeofday | toma el tiempo actual
CPU        | if( argc != 5 )      | Valida que los parametros de entrada se reciban correctamente
CPU        | rand       | obtiene numeros aleaotorios
CPU        | for...for       | Completa con valores aleatorios la matriz A
CPU        | for...for    | Completa con valores aleatorios la matriz B
CPU        | for...for...for       | Multiplica las matriz A con la matriz dejando el resultado en la matriz R de manera secuencial
CPU        | for...for...for       | Multiplica las matriz A con la matriz dejando el resultado en la matriz R, con OpenMP 
CPU        | for...for       | Muestro matriz A
CPU        | for...for       | Muestra matriz B
CPU        | for...for       | Muestra matriz R
CPU        | cout         | Muesta por pantalla los resultados de los tiempos de ejecución




# 5 Conclusiones

Como podemos observar en este ejercicio donde se realizó una multiplicacion de matrices, el uso del OpenMP no siempre nos dá una ventaja frente al procesamiento secuencial. Esto depende segun el tamaño de las matrices que se van a multiplicar. Por ejemplo si tenemos dos matrices de 2x2 que se van a multiplicar, uno de los tiempos que se llego a tomar para esta operacion es el siguiente:

*   Tiempo Sec : 0.000953674 [ms]
*   Tiempo Omp  : 0.13113 [ms]

Donde vemos claramente que el tiempo en la version secuencial es mucho mejor.

Sin embargo para entradas muy grandes, por ejemplo una multiplicacion de matrices de 100x100 podremos notar como hay una diferencia de tiempo a favor de OpenMp

*   Tiempo Sec  : 10.8368 [ms]
*   Tiempo Omp  : 9.79114 [ms]
}

Como conclusión podemos ver que en este ejemplo depende con que tamaño de matrices se está trabajando para decidir cual implementacion se va a utilizar. Si uno simpleemente hara el producto entre matrices pequeñas, entonces no es viable usar OpenMP ya que los tiempos de ejecucion muchos mayores. En cambio, si uno piensa realizar la multiplicacion de matrices de gran tamaño, quizas pueda considerar usar OpenMP.






# 6 Bibliografía

[1] LEY DE AMDAHL: [PDF](https://www.cartagena99.com/recursos/alumnos/apuntes/ININF1_M10_U1_T4_MT.pdf)

[2] Producto de matrices: [Referencia](https://www.superprof.es/apuntes/escolar/matematicas/algebralineal/matrices/producto-de-matrices.html)

[3] Algebra de matrices: [PDF](https://www.uv.mx/personal/aherrera/files/2014/08/08b.-MATRICES-2.pdf)

[4] OpenMP: [Referencia](http://www.bowdoin.edu/~ltoma/teaching/cs3225-GIS/fall16/Lectures/openmp.html#:~:text=OpenMP%20in%20a%20nutshell&text=Parallel%20code%20with%20OpenMP%20marks,and%20run%20the%20same%20code.)


